# STEELTRACE™ LAB — Ejecución MVP (Binder)

Este cuaderno ejecuta la tubería *stub* del MVP:
1) Ingesta/DQ → 2) SHACL → 3) RAGA → 4) EEE-Gate → 5) XBRL → 6) Evidencias (simulado)

**Nota:** Esta versión usa scripts de ejemplo (stubs) incluidos en el repo. Sirve para validar el *pipeline* y generar artefactos verificables para la consultora.

In [ ]:
import os, json, pathlib, sys, subprocess
print('Working dir:', os.getcwd())
print('Repo tree (top-level):')
print('\n'.join(os.listdir('.')))

## 1) Ingesta y DQ (MCP)

In [ ]:
!python scripts/mcp_ingest.py
print('\nOK: data/normalized y data/dq_report.json generados')

## 2) Validación semántica (SHACL — stub)

In [ ]:
!python scripts/shacl_validate.py
print(open('ontology/validation.log').read())

## 3) Razonamiento y KPIs (RAGA — stub)

In [ ]:
!python scripts/raga_compute.py
from pathlib import Path
print('KPIs:')
print(Path('raga/kpis.json').read_text())
print('\nExplain:')
print(Path('raga/explain.json').read_text())

## 4) Control de publicación (EEE-Gate — stub)

In [ ]:
!python scripts/eee_gate.py
from pathlib import Path
print(Path('ops/gate_report.json').read_text())
print('\nEEE report:')
print(Path('eee/eee_report.json').read_text())

## 5) Generación/Validación XBRL — stub

In [ ]:
!python scripts/xbrl_generate.py
from pathlib import Path
print('XBRL:')
print(Path('xbrl/informe.xbrl').read_text())
print('\nXBRL validation:')
print(Path('xbrl/validation.log').read_text())

## 6) Evidencias (Merkle + TSA simulada — stub)

In [ ]:
!bash scripts/evidence_sign.sh
from pathlib import Path
print('Manifest:')
print(Path('evidence/evidence_manifest.json').read_text())
print('\nVerificación TSA (simulada):')
print(Path('evidence/verify/2025Q1.txt').read_text())

## 7) Resumen de artefactos generados

In [ ]:
from pathlib import Path
paths = [
  'data/normalized/energy_2024-01.json',
  'ontology/validation.log',
  'raga/kpis.json', 'raga/explain.json',
  'ops/gate_report.json', 'eee/eee_report.json',
  'xbrl/informe.xbrl', 'xbrl/validation.log',
  'evidence/evidence_manifest.json', 'evidence/tokens/2025Q1.tsr'
]
for p in paths:
    print(p, '→', 'OK' if Path(p).exists() else 'MISSING')